In [3]:
import config as cfg
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from pathlib import Path
from sklearn.model_selection import train_test_split
import pickle

TypeError: Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

### Load the train, validation, test sets

In [2]:
data = pd.read_pickle(cfg.PATHS.TRANSFORMED_DATA_PICKLE)
with open(cfg.PATHS.EMBEDDINGS, 'rb') as f:
    embeddings = pickle.load(f)

In [ ]:
#concat (embeddings is an array)
data = pd.concat([data, pd.DataFrame(embeddings)], axis=1)
data.drop("text", axis=1, inplace=True)

In [4]:

# Split the data into training and testing sets
X = data.drop(columns=["useful", "funny", "cool"], axis=1)
y = data[["useful", "funny", "cool"]]

del data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

del X
del y

### Build the model

In [5]:
model = Sequential()
model.add(Dense(64, activation="relu", input_shape=(X_train.shape[1],)))
model.add(Dense(32, activation="relu"))
model.add(Dense(16, activation="relu"))
model.add(Dense(3, activation="softmax"))
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["acc"])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                109568    
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 16)                528       
                                                                 
 dense_3 (Dense)             (None, 3)                 51        
                                                                 
Total params: 112,227
Trainable params: 112,227
Non-trainable params: 0
_________________________________________________________________


In [6]:
history = model.fit(
    np.array(X_train),
    np.array(y_train),
    epochs=10,
    batch_size=32,
    validation_split=0.2,
)

Epoch 1/10


2023-04-14 10:50:06.349303: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


4463/4463 [==============================] - 5s 1ms/step - loss: 2723634432.0000 - acc: 0.8304 - val_loss: 11235980288.0000 - val_acc: 0.8320
Epoch 2/10
4463/4463 [==============================] - 4s 900us/step - loss: 45632724992.0000 - acc: 0.8304 - val_loss: 102464839680.0000 - val_acc: 0.8320
Epoch 3/10
4463/4463 [==============================] - 4s 942us/step - loss: 221939728384.0000 - acc: 0.8304 - val_loss: 390303514624.0000 - val_acc: 0.8320
Epoch 4/10
4463/4463 [==============================] - 4s 921us/step - loss: 665864241152.0000 - acc: 0.8304 - val_loss: 1029724831744.0000 - val_acc: 0.8320
Epoch 5/10
4463/4463 [==============================] - 4s 901us/step - loss: 1549175357440.0000 - acc: 0.8304 - val_loss: 2229544157184.0000 - val_acc: 0.8320
Epoch 6/10
4463/4463 [==============================] - 4s 912us/step - loss: 3098737115136.0000 - acc: 0.8304 - val_loss: 4201672605696.0000 - val_acc: 0.8320
Epoch 7/10
4463/4463 [==============================] - 4s 945us

In [7]:
model.evaluate(np.array(X_test), np.array(y_test))

1395/1395 [==============================] - 1s 415us/step - loss: 25770254663680.0000 - acc: 0.8322


[25770254663680.0, 0.8322018384933472]

In [8]:
# find indices of all rows in test data with greater than 10 votes in any category
model.predict(np.array(X_test))

1395/1395 [==============================] - 1s 349us/step


array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

In [9]:
pickle.dump(model, cfg.PATHS.KERAS_MODEL.open("wb"))